In [1]:
import os
import logging
working_directory = os.getcwd() + "/"
project_name = "app_vukwm_bag_delivery/"
if project_name in working_directory:
    base_directory = working_directory[:working_directory.find(project_name) + len(project_name)]
    os.chdir(base_directory)
    working_directory = os.getcwd() + "/"
    logging.info(f"Working directory changed to {working_directory}")
else:
    logging.warning(f"Project directory {project_name} not found in {working_directory}")

In [2]:
from keplergl import KeplerGl
import pandas as pd

In [9]:
assigned_stops =  pd.read_csv("data/local_test/04_View_Routes/assigned_stops.csv")
unassigned_jobs =  pd.read_csv("data/local_test/04_View_Routes/unassigned_jobs.csv")
unassigned_stops = pd.read_csv("data/local_test/04_View_Routes/unassigned_stops.csv")
unassigned_routes = pd.read_csv("data/local_test/04_View_Routes/unassigned_routes.csv")
unserviced_stops = pd.read_csv("data/local_test/04_View_Routes/unserviced_stops.csv")

In [22]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [
                {
                    "dataId": ["assigned_stops"],
                    "id": "6migbh5d7",
                    "name": ["arrival_time"],
                    "type": "timeRange",
                    "value": [1673427600000, 1673451419000],
                    "enlarged": True,
                    "plotType": "histogram",
                    "animationWindow": "free",
                    "yAxis": None,
                    "speed": 1,
                },
                {
                    "dataId": ["assigned_stops"],
                    "id": "9zf5366ym",
                    "name": ["route_id"],
                    "type": "multiSelect",
                    "value": [],
                    "enlarged": False,
                    "plotType": "histogram",
                    "animationWindow": "free",
                    "yAxis": None,
                    "speed": 1,
                },
            ],
            "layers": [
                {
                    "id": "30wwgdw",
                    "type": "point",
                    "config": {
                        "dataId": "assigned_stops",
                        "label": "Stop",
                        "color": [255, 153, 31],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "lat": "latitude",
                            "lng": "longitude",
                            "altitude": None,
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 25,
                            "fixedRadius": False,
                            "opacity": 0.75,
                            "outline": False,
                            "thickness": 2,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Uber Viz Qualitative 4",
                                "type": "qualitative",
                                "category": "Uber",
                                "colors": [
                                    "#12939A",
                                    "#DDB27C",
                                    "#88572C",
                                    "#FF991F",
                                    "#F15C17",
                                    "#223F9A",
                                    "#DA70BF",
                                    "#125C77",
                                    "#4DC19C",
                                    "#776E57",
                                    "#17B8BE",
                                    "#F6D18A",
                                    "#B7885E",
                                    "#FFCB99",
                                    "#F89570",
                                    "#829AE3",
                                    "#E79FD5",
                                    "#1E96BE",
                                    "#89DAC1",
                                    "#B3AD9E",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "radiusRange": [0, 50],
                            "filled": True,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": {"name": "job_sequence", "type": "integer"},
                                "color": [255, 255, 255],
                                "size": 14,
                                "offset": [0, 0],
                                "anchor": "middle",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "route_id", "type": "string"},
                        "colorScale": "ordinal",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                    },
                },
                {
                    "id": "vb8tg8f",
                    "type": "arc",
                    "config": {
                        "dataId": "assigned_stops",
                        "label": "Stop to road link",
                        "color": [146, 38, 198],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "lat0": "latitude",
                            "lng0": "longitude",
                            "lat1": "road_latitude",
                            "lng1": "road_longitude",
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.5,
                            "thickness": 3,
                            "colorRange": {
                                "name": "Uber Viz Qualitative 4",
                                "type": "qualitative",
                                "category": "Uber",
                                "colors": [
                                    "#12939A",
                                    "#DDB27C",
                                    "#88572C",
                                    "#FF991F",
                                    "#F15C17",
                                    "#223F9A",
                                    "#DA70BF",
                                    "#125C77",
                                    "#4DC19C",
                                    "#776E57",
                                    "#17B8BE",
                                    "#F6D18A",
                                    "#B7885E",
                                    "#FFCB99",
                                    "#F89570",
                                    "#829AE3",
                                    "#E79FD5",
                                    "#1E96BE",
                                    "#89DAC1",
                                    "#B3AD9E",
                                ],
                            },
                            "sizeRange": [0, 10],
                            "targetColor": None,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "route_id", "type": "string"},
                        "colorScale": "ordinal",
                        "sizeField": None,
                        "sizeScale": "linear",
                    },
                },
                {
                    "id": "dvqkcg",
                    "type": "geojson",
                    "config": {
                        "dataId": "assigned_stops",
                        "label": "Travel path",
                        "color": [218, 112, 191],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {"geojson": "travel_path_to_stop"},
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.5,
                            "thickness": 0.8,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "Uber Viz Qualitative 4",
                                "type": "qualitative",
                                "category": "Uber",
                                "colors": [
                                    "#12939A",
                                    "#DDB27C",
                                    "#88572C",
                                    "#FF991F",
                                    "#F15C17",
                                    "#223F9A",
                                    "#DA70BF",
                                    "#125C77",
                                    "#4DC19C",
                                    "#776E57",
                                    "#17B8BE",
                                    "#F6D18A",
                                    "#B7885E",
                                    "#FFCB99",
                                    "#F89570",
                                    "#829AE3",
                                    "#E79FD5",
                                    "#1E96BE",
                                    "#89DAC1",
                                    "#B3AD9E",
                                ],
                            },
                            "radius": 10,
                            "sizeRange": [0, 10],
                            "radiusRange": [0, 50],
                            "heightRange": [0, 500],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,
                            "filled": False,
                            "enable3d": False,
                            "wireframe": False,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": None,
                        "colorScale": "quantile",
                        "strokeColorField": {"name": "route_id", "type": "string"},
                        "strokeColorScale": "ordinal",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear",
                    },
                },
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "assigned_stops": [
                            {"name": "route_id", "format": None},
                            {"name": "profile", "format": None},
                            {"name": "stop_id", "format": None},
                            {"name": "job_sequence", "format": None},
                            {"name": "arrival_time", "format": None},
                            {"name": "service_start_time", "format": None},
                            {"name": "departure_time", "format": None},
                            {"name": "waiting_duration__seconds", "format": None},
                            {"name": "activity_type", "format": None},
                            {"name": "demand", "format": None},
                            {"name": "skills", "format": None},
                            {"name": "time_window_start", "format": None},
                            {"name": "time_window_end", "format": None},
                            {"name": "service_issue", "format": None},
                        ]
                    },
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True,
                },
                "brush": {"size": 0.5, "enabled": False},
                "geocoder": {"enabled": False},
                "coordinate": {"enabled": False},
            },
            "layerBlending": "normal",
            "splitMaps": [],
            "animationConfig": {"currentTime": None, "speed": 1},
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 51.49665988424509,
            "longitude": -0.1449888889219759,
            "pitch": 0,
            "zoom": 11.788588869841922,
            "isSplit": False,
        },
        "mapStyle": {
            "styleType": "dark",
            "topLayerGroups": {},
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False,
            },
            "threeDBuildingColor": [
                9.665468314072013,
                17.18305478057247,
                31.1442867897876,
            ],
            "mapStyles": {},
        },
    },
}


In [23]:
# config=m.config
# Point name Stop
# Road to arc Stop to road link
# Path Travel path

In [24]:
m = KeplerGl(data={"assigned_stops": assigned_stops}, height=750, config=config)
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['assigned_stops'], 'id': '6m…

In [10]:
from shapely import wkt
import app_vukwm_bag_delivery.osrm_get_routes as osrm_get_routes
import pandas as pd
def generate_route_data(assigned_stops, unassigned_routes):

    assigned_stops_route = assigned_stops.merge(
        unassigned_routes.drop(columns=["lat", "lon"])
    )


    assigned_stops_route = assigned_stops_route.assign(
        route_id=assigned_stops_route["Vehicle id"],
        latitude=assigned_stops_route["lat"],
        longitude=assigned_stops_route["lon"],
        vehicle_type=assigned_stops_route["Type"],
        stop_id=np.arange(0, assigned_stops_route.shape[0]),
    )

    unassigned_routes = unassigned_routes.assign(
    route_id=unassigned_routes["Vehicle id"],
    vehicle_type=unassigned_routes["Type"]
)
    osrm_routes = osrm_get_routes.return_route_osrm_info(unassigned_routes, assigned_stops_route)

    travel_legs = osrm_routes[0].copy()

    travel_legs["geometry"] = travel_legs["geometry"].apply(wkt.dumps)
    travel_legs = pd.DataFrame(travel_legs)

    travel_legs["route_sequence"] = travel_legs["travel_sequence"] + 1
    assigned_stops_route_paths = assigned_stops_route.merge(travel_legs, how="left")
    assigned_stops_route_paths["duration_seconds"] = assigned_stops_route_paths[
        "duration_seconds"
    ].fillna(0)
    assigned_stops_route_paths["distance_km"] = assigned_stops_route_paths[
        "distance_km"
    ].fillna(0)

    assigned_stops_route_paths["service_duration_seconds"] = (
        assigned_stops_route_paths["Average TAT per delivery (min)"]
        * 60
        * (~assigned_stops_route_paths["Ticket No"].isna())
    )

    assigned_stops_route_paths["travel_to_service_duration"] = (
        assigned_stops_route_paths["duration_seconds"]
        + assigned_stops_route_paths["service_duration_seconds"]
    )
    assigned_stops_route_paths["total_duration"] = assigned_stops_route_paths.groupby(
        ["route_id"]
    )["travel_to_service_duration"].cumsum()

    assigned_stops_route_paths["depart_stop_seconds"] = assigned_stops_route_paths[
        "total_duration"
    ]
    assigned_stops_route_paths["arrive_stop_seconds"] = (
        assigned_stops_route_paths["total_duration"]
        - assigned_stops_route_paths["service_duration_seconds"]
    )
    assigned_stops_route_paths["arrival_time"] = pd.to_datetime(
        assigned_stops_route_paths["Shift start time"]
    ) + pd.to_timedelta(assigned_stops_route_paths["arrive_stop_seconds"], unit="s")
    assigned_stops_route_paths["depart_time"] = pd.to_datetime(
        assigned_stops_route_paths["Shift start time"]
    ) + pd.to_timedelta(assigned_stops_route_paths["depart_stop_seconds"], unit="s")

    return assigned_stops_route_paths


In [11]:
assigned_stops_route_paths = generate_route_data(assigned_stops, unassigned_routes)

/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [48]:
import plotly.express as px
assigned_stops_route_paths_plot = assigned_stops_route_paths.copy()
assigned_stops_route_paths_plot = assigned_stops_route_paths_plot.rename(columns={"arrival_time": "Arrival time","depart_time": "Depart time", "route_id": "Route ID", "vehicle_type": "Vehicle type", "route_sequence": "Route sequence"})
fig = px.timeline(assigned_stops_route_paths_plot, x_start="Arrival time", x_end="Depart time", y="Route ID", color="Vehicle type", text="Route sequence")
fig.show()

In [34]:
import plotly.express as px
import pandas as pd

In [87]:
travel_legs = osrm_routes[0].copy()
travel_legs.shape[0]

119

In [88]:
travel_legs["geometry"] = travel_legs["geometry"].apply(wkt.dumps)
travel_legs = pd.DataFrame(travel_legs)

/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [63]:
assigned_stops_route.columns

Index(['Ticket No', 'Customer Bk', 'Site Bk', 'Site Name',
       'Transport Area Code', 'Product Name', 'Quantity', 'Created Date',
       'Required Date', 'Notes', 'on hold', 'Schedule ID', 'Scheduled Date',
       'Completed Date', 'Registration No', 'Site Address1', 'Site Address2',
       'Site Address3', 'Site Address4', 'Site Address5', 'Site Post Town',
       'Site Post Code', 'Site Latitude', 'Site Longitude', 'Site Address',
       'collection_date', 'tansport_area_num', 'lat', 'lon', 'Transport area',
       'Product description', 'Total boxes', 'route_sequence', 'Vehicle id',
       'Type', 'Capacity (#boxes)', 'Depot', 'Shift start time',
       'Average TAT per delivery (min)', 'Dedicated transport zones',
       'Cost (£) per km', 'Cost (£) per hour', 'route_id', 'latitude',
       'longitude', 'vehicle_type', 'stop_id'],
      dtype='object')

In [89]:
travel_legs["route_sequence"] = travel_legs["travel_sequence"] + 1
assigned_stops_route_paths = assigned_stops_route.merge(travel_legs, how="left")
assigned_stops_route_paths["duration_seconds"] = assigned_stops_route_paths[
    "duration_seconds"
].fillna(0)
assigned_stops_route_paths["distance_km"] = assigned_stops_route_paths[
    "distance_km"
].fillna(0)

assigned_stops_route_paths["service_duration_seconds"] = (
    assigned_stops_route_paths["Average TAT per delivery (min)"]
    * 60
    * (~assigned_stops_route_paths["Ticket No"].isna())
)

assigned_stops_route_paths["travel_to_service_duration"] = (
    assigned_stops_route_paths["duration_seconds"]
    + assigned_stops_route_paths["service_duration_seconds"]
)
assigned_stops_route_paths["total_duration"] = assigned_stops_route_paths.groupby(
    ["route_id"]
)["travel_to_service_duration"].cumsum()

assigned_stops_route_paths["depart_stop_seconds"] = assigned_stops_route_paths[
    "total_duration"
]
assigned_stops_route_paths["arrive_stop_seconds"] = (
    assigned_stops_route_paths["total_duration"]
    - assigned_stops_route_paths["service_duration_seconds"]
)
assigned_stops_route_paths["arrival_time"] = pd.to_datetime(
    assigned_stops_route_paths["Shift start time"]
) + pd.to_timedelta(assigned_stops_route_paths["arrive_stop_seconds"], unit="s")
assigned_stops_route_paths["depart_time"] = pd.to_datetime(
    assigned_stops_route_paths["Shift start time"]
) + pd.to_timedelta(assigned_stops_route_paths["depart_stop_seconds"], unit="s")


In [105]:
def return_order_config():
    config = {
        "version": "v1",
        "config": {
            "visState": {
                "filters": [
                    {
                        "dataId": ["orders"],
                        "id": "yt10dkouq",
                        "name": ["arrival_time"],
                        "type": "timeRange",
                        "value": [1671613200000, 1671626666000],
                        "enlarged": True,
                        "plotType": "histogram",
                        "animationWindow": "free",
                        "yAxis": None,
                        "speed": 1,
                    },
                    {
                        "dataId": ["orders"],
                        "id": "wrkhad0sp",
                        "name": ["Registration No"],
                        "type": "multiSelect",
                        "value": [],
                        "enlarged": False,
                        "plotType": "histogram",
                        "animationWindow": "free",
                        "yAxis": None,
                        "speed": 1,
                    },
                ],
                "layers": [
                    {
                        "id": "o4a1e2",
                        "type": "point",
                        "config": {
                            "dataId": "orders",
                            "label": "Stops",
                            "color": [241, 92, 23],
                            "highlightColor": [252, 242, 26, 255],
                            "columns": {
                                "lat": "Site Latitude",
                                "lng": "Site Longitude",
                                "altitude": None,
                            },
                            "isVisible": True,
                            "visConfig": {
                                "radius": 15,
                                "fixedRadius": False,
                                "opacity": 0.81,
                                "outline": True,
                                "thickness": 2,
                                "strokeColor": [25, 20, 16],
                                "colorRange": {
                                    "name": "Uber Viz Qualitative 4",
                                    "type": "qualitative",
                                    "category": "Uber",
                                    "colors": [
                                        "#12939A",
                                        "#DDB27C",
                                        "#88572C",
                                        "#FF991F",
                                        "#F15C17",
                                        "#223F9A",
                                        "#DA70BF",
                                        "#125C77",
                                        "#4DC19C",
                                        "#776E57",
                                        "#17B8BE",
                                        "#F6D18A",
                                        "#B7885E",
                                        "#FFCB99",
                                        "#F89570",
                                        "#829AE3",
                                        "#E79FD5",
                                        "#1E96BE",
                                        "#89DAC1",
                                        "#B3AD9E",
                                    ],
                                },
                                "strokeColorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": [
                                        "#5A1846",
                                        "#900C3F",
                                        "#C70039",
                                        "#E3611C",
                                        "#F1920E",
                                        "#FFC300",
                                    ],
                                },
                                "radiusRange": [0, 50],
                                "filled": True,
                            },
                            "hidden": False,
                            "textLabel": [
                                {
                                    "field": {
                                        "name": "route_sequence",
                                        "type": "integer",
                                    },
                                    "color": [205, 201, 191],
                                    "size": 18,
                                    "offset": [0, 0],
                                    "anchor": "start",
                                    "alignment": "center",
                                }
                            ],
                        },
                        "visualChannels": {
                            "colorField": {"name": "Registration No", "type": "string"},
                            "colorScale": "ordinal",
                            "strokeColorField": None,
                            "strokeColorScale": "quantile",
                            "sizeField": None,
                            "sizeScale": "linear",
                        },
                    },
                    {
                        "id": "h5ysd8s",
                        "type": "geojson",
                        "config": {
                            "dataId": "orders",
                            "label": "Routes",
                            "color": [255, 203, 153],
                            "highlightColor": [252, 242, 26, 255],
                            "columns": {"geojson": "geometry"},
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "strokeOpacity": 0.8,
                                "thickness": 0.5,
                                "strokeColor": None,
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": [
                                        "#5A1846",
                                        "#900C3F",
                                        "#C70039",
                                        "#E3611C",
                                        "#F1920E",
                                        "#FFC300",
                                    ],
                                },
                                "strokeColorRange": {
                                    "name": "Uber Viz Qualitative 4",
                                    "type": "qualitative",
                                    "category": "Uber",
                                    "colors": [
                                        "#12939A",
                                        "#DDB27C",
                                        "#88572C",
                                        "#FF991F",
                                        "#F15C17",
                                        "#223F9A",
                                        "#DA70BF",
                                        "#125C77",
                                        "#4DC19C",
                                        "#776E57",
                                        "#17B8BE",
                                        "#F6D18A",
                                        "#B7885E",
                                        "#FFCB99",
                                        "#F89570",
                                        "#829AE3",
                                        "#E79FD5",
                                        "#1E96BE",
                                        "#89DAC1",
                                        "#B3AD9E",
                                    ],
                                },
                                "radius": 10,
                                "sizeRange": [0, 10],
                                "radiusRange": [0, 50],
                                "heightRange": [0, 500],
                                "elevationScale": 5,
                                "enableElevationZoomFactor": True,
                                "stroked": True,
                                "filled": False,
                                "enable3d": False,
                                "wireframe": False,
                            },
                            "hidden": False,
                            "textLabel": [
                                {
                                    "field": None,
                                    "color": [255, 255, 255],
                                    "size": 18,
                                    "offset": [0, 0],
                                    "anchor": "start",
                                    "alignment": "center",
                                }
                            ],
                        },
                        "visualChannels": {
                            "colorField": None,
                            "colorScale": "quantile",
                            "strokeColorField": {
                                "name": "Registration No",
                                "type": "string",
                            },
                            "strokeColorScale": "ordinal",
                            "sizeField": None,
                            "sizeScale": "linear",
                            "heightField": None,
                            "heightScale": "linear",
                            "radiusField": None,
                            "radiusScale": "linear",
                        },
                    },
                ],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "orders": [
                                {"name": "Customer Bk", "format": None},
                                {"name": "Site Bk", "format": None},
                                {"name": "Site Name", "format": None},
                                {"name": "Transport Area Code", "format": None},
                                {"name": "Site Address", "format": None},
                                {"name": "Notes", "format": None},
                                {"name": "Product description", "format": None},
                                {"name": "Registration No", "format": None},
                                {"name": "route_sequence", "format": None},
                                {"name": "Transport area", "format": None},
                                {"name": "Type", "format": None},
                                {"name": "arrival_time", "format": None},
                                {"name": "depart_time", "format": None},
                            ]
                        },
                        "compareMode": False,
                        "compareType": "absolute",
                        "enabled": True,
                    },
                    "brush": {"size": 0.5, "enabled": False},
                    "geocoder": {"enabled": False},
                    "coordinate": {"enabled": False},
                },
                "layerBlending": "normal",
                "splitMaps": [],
                "animationConfig": {"currentTime": None, "speed": 1},
            },
            "mapState": {
                "bearing": 0,
                "dragRotate": False,
                "latitude": 51.503679000596435,
                "longitude": -0.11381729292045753,
                "pitch": 0,
                "zoom": 12.5,
                "isSplit": False,
            },
            "mapStyle": {
                "styleType": "dark",
                "topLayerGroups": {},
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": False,
                    "building": True,
                    "water": True,
                    "land": True,
                    "3d building": False,
                },
                "threeDBuildingColor": [
                    9.665468314072013,
                    17.18305478057247,
                    31.1442867897876,
                ],
                "mapStyles": {},
            },
        },
    }
    return config


In [106]:
routes_plot = assigned_stops_route_paths.copy()

In [107]:
m = Kepler(data={"orders": routes_plot}, config=return_order_config(), height=750)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [108]:
m.get_render()

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['orders'], 'id': 'yt10dkouq'…

In [5]:
from app_vukwm_bag_delivery.return_routes import generate_route_data
import pandas as pd